In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_colwidth', None)
import os
home_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
from tqdm import tqdm



index_selected_list = ['上证50','上证150','沪深300','中证500','中证1000','科创50','创业板50','创业300']
ignored_list = ['上证180','上证380','深证100','深证200','深证700']
concepts_list = pd.read_csv(home_path + '/static/concepts_checklist.csv',encoding="utf-8")
industry_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['行业','产品','业务','行业+']) & concepts_list['active']==1 ]['概念名称'].values.tolist()
events_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['事件','改革','政策']) & concepts_list['active']==1]['概念名称'].values.tolist()
master_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['知名企业','国企体系']) & concepts_list['active']==1]['概念名称'].values.tolist()
exclude_concepts_list = concepts_list.loc[concepts_list['分类'].isin(['交易'])]['概念名称'].values.tolist()

wc_result = pd.DataFrame()

for i in tqdm(range(1,2)):
    print('page: '+ str(i))
    sr = wc.search('2023年预测净利润 总股本 行业 所属指数类 公司亮点',page =i).fillna('')
    sr['沪深指数'] = ''
    sr['行业概念'] = ''
    sr['事件概念'] = ''
    sr['体系概念'] = ''

    for row, l in sr.iterrows():
        # sr['主营产品名称'][row] = ','.join(l['主营产品名称'].split('||'))
        sr['沪深指数'][row] = ','.join(set(l['所属指数类'].split(';')).intersection(index_selected_list))
        sr['所属概念'][row] = ','.join(set(l['所属概念'].split(';')).difference(exclude_concepts_list))
        # sr['行业概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(industry_concepts_list)).replace('金属','')
        # sr['事件概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(events_concepts_list))
        # sr['体系概念'][row] = ','.join(set(l['所属概念'].replace('概念股','').replace('概念','').split(';')).intersection(master_concepts_list))

    sr['所属概念'] =   sr['所属概念'].str.replace('概念股','').str.replace('概念','')
#     sr['流动市值'] = (pd.to_numeric(sr['a股市值(不含限售股)'])/100000000).round(1).fillna('')    
    

In [176]:
import requests

url = 'http://www.iwencai.com/gateway/urp/v7/landing/getDataList'

query_string = '所属概念 公司亮点 所属同花顺行业 所属指数类'
page = 3

condition = []
condition_1 = {
        'score': 0.0,
        'chunkedResult': '_&_'.join(query_string.split(' ')),
        'opName': 'and',
        'opProperty': '',
        'sonSize': 3,
        'relatedSize': '0',
        'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
        'source': 'text2sql'
        }
condition.append(condition_1)

for i in query_string.split(' '):
#     i = i.replace('指数','指数类')
    dic = {
        'dateText': '',
        'indexName': i,
        'indexProperties': [],
        'child_node': i,
        'ci': True,
        'source': 'text2sql',
        'type': 'index',
        'indexPropertiesMap': {},
        'reportType': 'null',
        'ciChunk': i,
        'createBy': 'preCache',
        'uiText': i,
        'valueType': '_' + i,
        'domain': 'abs_股票领域',
        'sonSize': 0,
        'dateList': []
        }
    condition.append(dic)
condition = str(condition).replace("'", '"').replace(' ','')


payload = {
    "question": query_string,
    "page": page,
    "perpage": 100,
    "source": "Ths_iwencai_Xuangu",
    'urp_sort_way': 'desc',
    'urp_sort_index': '最新涨跌幅',
    'addheaderindexes': '',
    'codelist': '',
    'indexnamelimit': '',
    'ret': 'json_all',
    'urp_use_sort': '1',
    'user_id': '570662655',
    'uuids[0]': '24087',
    'query_type': 'stock',
    'comp_id': '6734520',
    'business_cat': 'soniu',
    'uuid': '24087',
#    以下为每次变化
#     'date_range[0]': '20230610',
#     'iwc_token': '0ac9529816863435305771116',
#     'sessionid': '9f98f2743c41f01ae1e20f42ce38d2a1',
#     'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
    'condition': str(condition).replace("'", '"').replace(' ','')
}

cookies = {
    "WafStatus":'0',
    "ta_random_userid":'anro5tt7uu',
    "cid":'e77a8079986b06e6a9a46e614cf7eee21659209895',
    "ComputerID":'e77a8079986b06e6a9a46e614cf7eee21659209895',
    "other_uid":'Ths_iwencai_Xuangu_x25492z942nd2ag2aqbfqwh7m852m7vi',
    "ttype":'WEB',
    "wencai_pc_version":'1',
    "user_status":'0',
    "user":'MDpteF81NzA2NjI2NTU6Ok5vbmU6NTAwOjU4MDY2MjY1NTo3LDExMTExMTExMTExLDQwOzQ0LDExLDQwOzYsMSw0MDs1LDEsNDA6MTY6Ojo1NzA2NjI2NTU6MTY4NjMyMTg1NTo6OjE2MTQ4OTgzMjA6NjA0ODAwOjA6MTk0Yzg1NGMxMzdlYzY1ZmY1NTcxNjIzMWU2YzA2MGZkOmRlZmF1bHRfNDow',
    "userid":'570662655',
    "u_name":'mx_570662655',
    "escapename":'mx_570662655',
    "ticket":'106f915b0a9985030a58c2151f69d9b8',
    "utk":'86be216097cbd990b074d0a691f39d39',
#     'THSSESSID':'210fbd845858470fd3dcc8c227',
#     "PHPSESSID":'i1jarg6eiikd16j6ajcjuqhb7j1a88je',
#     "v":'A1hilGtTiE5eR6Q8zbqmwNg1KY3vQa3eHoyRWZLcpY9GCvSzOlGMW261YLTh',
}


headers = {
    'Accept':'application/json, text/plain, */*',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,en-US;q=0.7,ja;q=0.6,zh-TW;q=0.5,nl;q=0.4',
    'Cache-Control':'no-cache',
    'Connection':'keep-alive',
    'Content-Length':'2221',
    'Content-Type':'application/x-www-form-urlencoded',
#     'Hexin-V':'A1hilGtTiE5eR6Q8zbqmwNg1KY3vQa3eHoyRWZLcpY9GCvSzOlGMW261YLTh',
    'Host':'www.iwencai.com',
    'Origin':'http://www.iwencai.com',
    'Pragma':'no-cache',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}



r = requests.post(url, data = payload, cookies = cookies, headers = headers)

import json
import pandas as pd
df = pd.DataFrame(r.json()['answer']['components'][0]['data']['datas'])
df

,股票代码,股票简称,最新价,最新涨跌幅,所属概念,公司亮点,所属同花顺行业,所属指数类,market_code,code
0,688190.SH,云路股份,70.69,5.697,融资融券;转融券标的;专精特新;无线充电;新能源汽车;地方国企改革;央企国企改革;国企改革;...,公司非晶合金材料的市场份额为全球领先，是非晶合金材料行业的龙头企业,有色金属-金属新材料-其他金属新材料,A股指数;上证指数;上证流通;中型综指;中证A股;中证流通;同花顺全A;工业指数;新综指,17,688190
1,300397.SZ,天和防务,10.95,5.695,超材料概念;军民融合;雷达;通用航空;安防;华为概念;芯片概念;转融券标的;5G;机器人概念...,在军民融合综合电子信息领域，拥有先进的技术、成熟的生产工艺,国防军工-国防军工-军工电子,中小创新;中证1000;中证1000USD;中证A股;中证全指;中证流通;创业300;创业创...,33,300397
2,688506.SH,百利天恒,78.74,5.691,转融券标的;中医药;创新药;融资融券;新股与次新股;科创次新股,全麻药丙泊酚国内市占率排名居前，黄芪颗粒国内样本医院中市占率第一,医药生物-化学制药-化学制剂,同花顺全A,17,688506
3,002008.SZ,大族激光,25.12,5.679,深紫外光;智能终端;国家科技大会;光纤;央视财经50;3D打印;融资融券;全面屏;苹果概念;...,亚洲最大激光加工设备生产商,机械设备-自动化设备-激光设备,300 ESG;300精明;300精明HKD;300精明USD;SME创新;专用指数;中创1...,33,002008
4,002891.SZ,中宠股份,24.64,5.660,深股通;人民币贬值受益;宠物经济;融资融券;转融券标的,全球宠物食品领域多元化品牌运营商,农林牧渔-农产品加工-饲料,中小创新;中小基础;中小综指;中证1000;中证1000USD;中证A股;中证全指;中证流通...,33,002891
...,...,...,...,...,...,...,...,...,...,...
95,688126.SH,沪硅产业,21.23,4.581,融资融券;转融券标的;芯片概念;国家大基金持股;中芯国际概念;MSCI概念;集成电路概念;沪股通,曾荣获国家科学技术进步一等奖、中国科学院杰出科技成就奖,电子-半导体及元件-半导体材料,300 ESG;300精明;300精明HKD;300精明USD;A股指数;上证中小;上证全指...,17,688126
96,002101.SZ,广东鸿图,19.00,4.568,创投;横琴新区;华为概念;5G;蔚来汽车概念;新能源汽车;特斯拉;华为汽车;汽车电子;宁德时...,国内少有的自研免热合金材料的压铸龙头，一体化压铸成型技术全球领先，成功下线国产最大尺寸680...,交运设备-汽车零部件-汽车零部件Ⅲ,中创400;中创500;中小300;中小创新;中小基础;中小综指;中证A股;中证全指;中证流...,33,002101
97,002531.SZ,天顺风能,15.58,4.564,新能源;深股通;融资融券;转融券标的;富时罗素概念;富时罗素概念股;标普道琼斯A股;风电;海...,中国最具规模的风力发电塔架专业制造企业之一,电力设备-电力设备-风电设备,中创400;中创500;中小100;中小300;中小基础;中小综指;中证500;中证500H...,33,002531
98,688687.SH,凯因科技,27.50,4.563,融资融券;转融券标的;生物医药;新冠治疗;肝炎概念;创新药,国内首家成功开发出丙肝高治愈率泛基因型全口服药物组合的医药企业,医药生物-生物制品-其他生物制品,A股指数;上证指数;上证流通;中型综指;中证A股;中证全指;中证流通;同花顺全A;工业指数;新综指,17,688687


In [165]:
a = '[{"score":0.0,"chunkedResult":"所属概念_&_所属指数_&_公司亮点","opName":"and","opProperty":"","sonSize":3,"relatedSize":"0","logid":"9f98f2743c41f01ae1e20f42ce38d2a1","source":"text2sql"},{"dateText":"","indexName":"所属概念","indexProperties":[],"child_node":"所属概念","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属概念","createBy":"preCache","uiText":"所属概念","valueType":"_所属概念","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"所属指数类","indexProperties":[],"child_node":"所属指数类","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属指数","createBy":"preCache","uiText":"所属指数类","valueType":"_所属指数类","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"公司亮点","indexProperties":[],"child_node":"公司亮点","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"公司亮点","createBy":"preCache","uiText":"公司亮点","valueType":"_公司亮点","domain":"abs_股票领域","sonSize":0,"dateList":[]}]'


In [131]:
query_string = '所属概念 所属指数 公司亮点 所属行业'
'_&_'.join(query_string.split(' '))

'所属概念_&_所属指数_&_公司亮点_&_所属行业'

In [167]:
a

'[{"score":0.0,"chunkedResult":"所属概念_&_所属指数_&_公司亮点","opName":"and","opProperty":"","sonSize":3,"relatedSize":"0","logid":"9f98f2743c41f01ae1e20f42ce38d2a1","source":"text2sql"},{"dateText":"","indexName":"所属概念","indexProperties":[],"child_node":"所属概念","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属概念","createBy":"preCache","uiText":"所属概念","valueType":"_所属概念","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"所属指数类","indexProperties":[],"child_node":"所属指数类","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属指数","createBy":"preCache","uiText":"所属指数类","valueType":"_所属指数类","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"公司亮点","indexProperties":[],"child_node":"公司亮点","ci":true,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"公司亮点","createBy":"preCache","uiText":"公司亮点","valueType":"_公司亮点","domain"

In [170]:
str(condition).replace("'", '"').replace(' ','')

'[{"score":0.0,"chunkedResult":"所属概念_&_所属指数_&_公司亮点","opName":"and","opProperty":"","sonSize":3,"relatedSize":"0","logid":"9f98f2743c41f01ae1e20f42ce38d2a1","source":"text2sql"},{"dateText":"","indexName":"所属概念","indexProperties":[],"child_node":"所属概念","ci":True,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属概念","createBy":"preCache","uiText":"所属概念","valueType":"_所属概念","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"所属指数","indexProperties":[],"child_node":"所属指数","ci":True,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"所属指数","createBy":"preCache","uiText":"所属指数","valueType":"_所属指数类","domain":"abs_股票领域","sonSize":0,"dateList":[]},{"dateText":"","indexName":"公司亮点","indexProperties":[],"child_node":"公司亮点","ci":True,"source":"text2sql","type":"index","indexPropertiesMap":{},"reportType":"null","ciChunk":"公司亮点","createBy":"preCache","uiText":"公司亮点","valueType":"_公司亮点","domain":"a

In [ ]:



{'score': 0.0,
  'chunkedResult': '_&_'.join(query_string.split(' ')),
  'opName': 'and',
  'opProperty': '',
  'sonSize': 3,
  'relatedSize': '0',
#   'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
  'source': 'text2sql'}


{'dateText': '',
  'indexName': '所属概念',
  'indexProperties': [],
  'child_node': '所属概念',
  'ci': True,
  'source': 'text2sql',
  'type': 'index',
  'indexPropertiesMap': {},
  'reportType': 'null',
  'ciChunk': '所属概念',
  'createBy': 'preCache',
  'uiText': '所属概念',
  'valueType': '_所属概念',
  'domain': 'abs_股票领域',
  'sonSize': 0,
  'dateList': []},

In [128]:
json.loads(a[0])

[{'score': 0.0,
  'chunkedResult': '所属概念_&_所属指数_&_公司亮点',
  'opName': 'and',
  'opProperty': '',
  'sonSize': 3,
  'relatedSize': '0',
  'logid': '9f98f2743c41f01ae1e20f42ce38d2a1',
  'source': 'text2sql'},
 {'dateText': '',
  'indexName': '所属概念',
  'indexProperties': [],
  'child_node': '所属概念',
  'ci': True,
  'source': 'text2sql',
  'type': 'index',
  'indexPropertiesMap': {},
  'reportType': 'null',
  'ciChunk': '所属概念',
  'createBy': 'preCache',
  'uiText': '所属概念',
  'valueType': '_所属概念',
  'domain': 'abs_股票领域',
  'sonSize': 0,
  'dateList': []},
 {'dateText': '',
  'indexName': '所属指数类',
  'indexProperties': [],
  'child_node': '所属指数类',
  'ci': True,
  'source': 'text2sql',
  'type': 'index',
  'indexPropertiesMap': {},
  'reportType': 'null',
  'ciChunk': '所属指数',
  'createBy': 'preCache',
  'uiText': '所属指数类',
  'valueType': '_所属指数类',
  'domain': 'abs_股票领域',
  'sonSize': 0,
  'dateList': []},
 {'dateText': '',
  'indexName': '公司亮点',
  'indexProperties': [],
  'child_node': '公司亮点',
  